In [1]:
import ee
import geemap
import os

import rasterio
import numpy as np
from PIL import Image

ee.Authenticate()
ee.Initialize()

In [ ]:
# ==========================================
# 1. LOAD DHS CLUSTERS
# ==========================================

dhs_shp_path = '/Users/ruben/Desktop/Thesis/TrainingData/PH_DHS_GPS/ph_dhs_sample_50.shp' 
dhs_points = geemap.shp_to_ee(dhs_shp_path)

In [ ]:
# ==========================================
# 2. DEFINE BUFFER FUNCTION
# ==========================================
# We need to turn each single point into a square box for the CNN.
# 2km buffer for urban, 5km for rural

def adaptive_buffer(feature):
    # 1. Get the Urban/Rural flag from column 'URBAN_RURA'
    urban_rural_status = ee.String(feature.get('URBAN_RURA'))
    
    # 2. Check if it is Urban (Compare string to 'U')
    is_urban = urban_rural_status.compareTo('U').eq(0)
    
    # 3. Choose Radius based on status
    # Logic: If Urban -> 2000m, Else -> 5000m
    radius = ee.Number(ee.Algorithms.If(is_urban, 2000, 5000))
    
    # 4. Buffer and Box
    return feature.buffer(radius).bounds()

# Apply the buffer function to all DHS points
dhs_squares = dhs_points.map(adaptive_buffer)

# Verify on Map
Map = geemap.Map()
Map.centerObject(dhs_points, 6)
Map.addLayer(dhs_points, {'color': 'red'}, 'Original DHS Points')
Map.addLayer(dhs_squares, {'color': 'blue'}, 'Buffered Squares (Download Area)')
Map

Map(center=[12.154454545945676, 122.42655436359965], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# ==========================================
# 3. DOWNLOAD CENTERED CHIPS
# ==========================================
output_dir = os.path.expanduser('/Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/dhs_sample_tiles')

print(f"Downloading {dhs_squares.size().getInfo()} clusters...")

# ==========================================
# 4. PREPARE SATELLITE IMAGE
# ==========================================
# Use a relaxed cloud filter to avoid "Black Images"
def mask_s2_clouds(image):
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(mask).divide(10000)

s2_image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-01-01', '2023-12-31') # 2023 calendar year
    .filterBounds(dhs_squares)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) # Allow 30% clouds
    .map(mask_s2_clouds)
    .median()
    .select(['B4', 'B3', 'B2', 'B8'])
    .clip(dhs_squares))

geemap.download_ee_image_tiles_parallel(
    image=s2_image,
    out_dir=output_dir,
    features=dhs_squares,
    scale=30, #30 m resolution
    crs='EPSG:3857'
    prefix='dhs_'
)

/opt/miniconda3/lib/python3.13/site-packages/geemap/common.py:12434: FutureWarning: 'BaseImage' is deprecated and will be removed in a future release.  Please use the 'ee.Image.gd' accessor instead.
  img = gd.download.BaseImage(image)
/opt/miniconda3/lib/python3.13/site-packages/geemap/common.py:12434: FutureWarning: 'BaseImage' is deprecated and will be removed in a future release.  Please use the 'ee.Image.gd' accessor instead.
  img = gd.download.BaseImage(image)
/opt/miniconda3/lib/python3.13/site-packages/geemap/common.py:12434: FutureWarning: 'BaseImage' is deprecated and will be removed in a future release.  Please use the 'ee.Image.gd' accessor instead.
  img = gd.download.BaseImage(image)
/opt/miniconda3/lib/python3.13/site-packages/geemap/common.py:12434: FutureWarning: 'BaseImage' is deprecated and will be removed in a future release.  Please use the 'ee.Image.gd' accessor instead.
  img = gd.download.BaseImage(image)
/opt/miniconda3/lib/python3.13/site-packages/geemap/comm

/opt/miniconda3/lib/python3.13/site-packages/geemap/common.py:12434: FutureWarning: 'BaseImage' is deprecated and will be removed in a future release.  Please use the 'ee.Image.gd' accessor instead.
  img = gd.download.BaseImage(image)
100%|██████████|1/1 tiles [00:03<00:00]
/opt/miniconda3/lib/python3.13/site-packages/geedim/image.py:254: RuntimeWarning: Couldn't find STAC entry for: 'None'.
  return STACClient().get(self.id)


100%|██████████|1/1 tiles [00:03<00:00]
/opt/miniconda3/lib/python3.13/site-packages/geedim/image.py:254: RuntimeWarning: Couldn't find STAC entry for: 'None'.
  return STACClient().get(self.id)


100%|██████████|1/1 tiles [00:04<00:00]
/opt/miniconda3/lib/python3.13/site-packages/geedim/image.py:254: RuntimeWarning: Couldn't find STAC entry for: 'None'.
  return STACClient().get(self.id)


100%|██████████|1/1 tiles [00:05<00:00]
/opt/miniconda3/lib/python3.13/site-packages/geedim/image.py:254: RuntimeWarning: Couldn't find STAC entry for: 'None'.
  return STACClient().get(self.id)


...

100%|██████████|1/1 tiles [00:06<00:00]


100%|██████████|1/1 tiles [00:15<00:00]
100%|██████████|1/1 tiles [00:05<00:00]


100%|██████████|1/1 tiles [00:07<00:00]
100%|██████████|1/1 tiles [00:05<00:00]
100%|██████████|1/1 tiles [00:05<00:00]
100%|██████████|1/1 tiles [00:36<00:00]
100%|██████████|1/1 tiles [00:06<00:00]
100%|██████████|1/1 tiles [00:06<00:00]
100%|██████████|1/1 tiles [00:08<00:00]
100%|██████████|1/1 tiles [00:12<00:00]
100%|██████████|1/1 tiles [00:12<00:00]


Finished in 89.21064805984497 seconds.


100%|██████████|1/1 tiles [00:19<00:00]


In [ ]:
# ==========================================
# 5. GENERATE JPG PREVIEWS
# ==========================================

# Path to TIF files
input_dir = os.path.expanduser('/Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/dhs_sample_tiles')

# Output directory for JPG previews
output_dir = os.path.join(input_dir, 'previews_jpg')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Reading TIFs from: {input_dir}")
print(f"Saving JPGs to:    {output_dir}")

# 2. DEFINING THE STRETCH FUNCTION
def stretch_band(band):
    # Calculate the 2nd and 98th percentiles (ignores the absolute outliers)
    p2, p98 = np.percentile(band, (2, 98))
    
    # Stretch the data so p2 becomes 0 (Black) and p98 becomes 255 (White)
    # This "pulls" the dark details out of the shadows
    img_rescale = np.interp(band, (p2, p98), (0, 255)).astype(np.uint8)
    return img_rescale

# 3. PROCESSING LOOP
tif_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

for filename in tif_files:
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename.replace('.tif', '.jpg'))
    
    try:
        with rasterio.open(input_path) as src:
            # Read RGB (Bands 1, 2, 3)
            r = src.read(1)
            g = src.read(2)
            b = src.read(3)
            
            # Check if empty (all zeros)
            if np.max(r) == 0:
                print(f"Skipping {filename}: Image is completely empty.")
                continue

            # Apply the stretch to each color channel independently
            r_str = stretch_band(r)
            g_str = stretch_band(g)
            b_str = stretch_band(b)
            
            # Stack and Save
            rgb = np.dstack((r_str, g_str, b_str))
            img = Image.fromarray(rgb)
            img.save(output_path, quality=85)
            
    except Exception as e:
        print(f"Error on {filename}: {e}")

print("-" * 30)
print(f"Done! Check the folder: {output_dir}")

Reading TIFs from: /Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/dhs_sample_tiles
Saving JPGs to:    /Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/dhs_sample_tiles/previews_jpg
------------------------------
Done! Check the folder: /Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/dhs_sample_tiles/previews_jpg
